In [ ]:
from pathlib import Path 
# scenes = ["scene0000_00", "scene0000_01", "scene0000_02", "scene0001_00", "scene0001_01", "scene0002_00",  "scene0002_01", 
          
scenes = ["scene0003_00"]#,"scene0000_01", "scene0000_02" ] #, "scene0004_00", "scene0005_00", "scene0005_01"]
    
    
ls = [str(s) for s in Path( f"/home/jonfrey/Datasets/scannet/scans/" ).rglob("*.jpg") 
  if str(s).find("color") != -1 and
      str(s).split('/')[-3] in scenes and
     int((str(s).split('/')[-1][:-4]))% 10 == 0   ]##


ls.sort(key= lambda x: 1000000*int(str(x).split('/')[-3][5:9]) +
        10000*int(str(x).split('/')[-3][-2:])+
        int(x.split('/')[-1][:-4]))

# ls = ls[int(len(ls)*0.8):]

import os
import sys 
os.chdir(os.path.join(os.getenv('HOME'), 'ASL'))
sys.path.insert(0, os.getcwd())
sys.path.append(os.path.join(os.getcwd() + '/src'))
from utils_asl import LabelLoaderAuto
import numpy as np
label_loader = LabelLoaderAuto(root_scannet= "/home/jonfrey/Datasets/scannet", confidence=0)

nr_labels = 7

paths = np.zeros( (len(ls),nr_labels), dtype=object )

names = ["gt" ]# , "labels_pretrain25k_withoutNaN"] #, "pretrain25k_", "detectron_stored_fast_loading", "labels_pretrain25k_withoutNaN"]

valid = []
for i, l in enumerate( ls ):
    paths[i,0] = l.replace('color','label-filt').replace('jpg','png')

    base = "/home/jonfrey/Datasets/labels_generated/"
    scene = l.split('/')[-3]
    idx = l.split('/')[-1][:-4]

    idf = "labels_pretrain25k_correct_mapping"
    paths[i,1] = os.path.join( base, idf, "scans", scene, idf , idx+ '.png'); names.append(idf)

    idf = "labels_pretrain25k_correct_mapping_reprojected"
    paths[i,2] = os.path.join( base, idf, "scans", scene, idf , idx+ '.png'); names.append(idf)
    
    idf = "labels_pcmr_oracle"
    paths[i,3] = os.path.join( base, idf, "scans", scene, idf , idx+ '.png'); names.append(idf)
    
    idf = "labels_pcmr_confidence_05_fixed_epochs"
    paths[i,4] = os.path.join( base, idf, "scans", scene, idf , idx+ '.png'); names.append(idf)

    idf = "labels_pcmr_confidence_05_fixed_epochs_reprojected"
    paths[i,5] = os.path.join( base, idf, "scans", scene, idf , idx+ '.png'); names.append(idf)
    
    idf = "debug_reprojected"
    paths[i,6] = os.path.join( base, idf, "scans", scene, idf , idx+ '.png'); names.append(idf)
    
    
    t = True
    for n in range( nr_labels ):
        if not os.path.isfile(paths[i,n]):
            print( "Not found: ", paths[i,n])
            t = False
    valid.append(t)

print(" Not found:", (np.array( valid ) == False).sum() , " Found: ", np.array( valid ).sum(), len(ls) ) 

In [ ]:
from visu import Visualizer
import imageio
visu_active = True
visu = Visualizer(os.getenv('HOME')+'/tmp', logger=None, epoch=0, store=True, num_classes=41)

In [ ]:
import time
st = time.time()

def f( paths):
    nr_labels = paths.shape[1]
    label_loader = LabelLoaderAuto(root_scannet= "/home/jonfrey/Datasets/scannet", confidence=0.9)
    
    acc1 = [ [] for l in range( nr_labels ) ]
    acc2 = [ [] for l in range( nr_labels ) ]
    for i in range(paths.shape[0]):
        print(i)
        gt,mode = label_loader.get(paths[i,0])
        
        for j in range( 1, paths.shape[1] ):
            try:
                out, _  = label_loader.get(paths[i,j])
            except:
                out = np.zeros_like(gt)
            m = gt != -1
            
            if m.sum() != 0:            
                ac = (gt[m] == out[m]).sum() / m.sum()
                acc1[j].append( float( ac ) )
                
            m = (gt != 0) * ( out != 0 )
            
            if m.sum() != 0:
                ac = (gt[m] == out[m]).sum() / m.sum()
                acc2[j].append( float( ac ) )

            if visu_active and (j == 1 or j == 6  ):
                print( j, names[j])
                img = imageio.imread( ls[i] )
                visu.plot_detectron( img = img, label = out, tag='test', jupyter=True, store=False,alpha=0.6)

    return acc1 ,acc2

acc1 ,acc2 = f(paths)
print(time.time()-st)

In [ ]:
print("Val Acc Task 4 \n ")

import numpy as np
for j in range(nr_labels):
    print( "GT-MASK", names[j], ": ", np.round( np.array( acc1[j] ).mean(),3))
    
print("")
for j in range(nr_labels):    
    print( "OUTPUT-LABELS", names[j],  ": ", np.round( np.array( acc2[j] ).mean(),3))
    
print("\n\nExplanation Difference in Numbers: -> Performing Scaling not Cropping ")

In [ ]:
label_loader = LabelLoaderAuto(root_scannet= "/home/jonfrey/Datasets/scannet", confidence=0.7)

label, _ = label_loader.get("/home/jonfrey/Datasets/labels_generated/label_pretrain25k_reprojected/scene0000_00/label_pretrain25k_reprojected/map_probs_with_confidence_0.5/0.png")
visu.plot_detectron( img = np.ones( (label.shape[0],label.shape[1],3)), label = label, tag='test', jupyter=True, store=False,alpha=0.3)

In [ ]:
import PIL 
res = PIL.Image.open("/home/jonfrey/Datasets/labels_generated/labels_pretrain25k_correct_mapping/scans/scene0005_01/labels_pretrain25k_correct_mapping/1260.png")